In [10]:
import requests
from bs4 import BeautifulSoup
import json

url = 'https://stats.espncricinfo.com/ci/engine/records/team/match_results.html?id=15174;type=tournament'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [11]:
# Parser Code
links = []
table_rows = soup.select('table.engineTable > tbody > tr.data1')
for row in table_rows:
    match_url = "https://www.espncricinfo.com" + row.select('td')[6].select('a')[0]['href']
    links.append(match_url)

batting_summary = []

In [12]:

for url in links:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    match = soup.find('span', text='MATCH FLOW').parent.parent.next_sibling.find_all_next('div')
    team1 = match[0].select('span')[0].text.replace(" innings","")
    team2 = match[1].select('span')[0].text.replace(" innings","")
    match_info = team1 + ' Vs ' + team2
    tables = soup.select('div > table.ci-scorecard-table')
    first_inning_rows = [row for row in tables[0].select('tbody > tr') if len(row.select('td')) >= 8]
    second_inning_rows = [row for row in tables[1].select('tbody > tr') if len(row.select('td')) >=8]
    summary = []
    try:
        
        for i in range(0,len(first_inning_rows)):
            # print(len(first_inning_rows))
            row = first_inning_rows[i]

            cols = row.select('td')
            summary.append({
                'match': match_info,
                'teamInnings': team1,
                'battingPos': i+1,
                'battername': cols[0].select('a > span > span')[0].text.replace('\xa0', ''),
                'dismissal': cols[1].text,
                'runs': cols[2].select('strong')[0].text,
                'balls': cols[3].text,
                '4s': cols[5].text,
                '6s': cols[6].text,
                'SR': cols[7].text
            })
           
    except Exception as e:
        print(e)
    
    try:        
        for i in range(0,len(second_inning_rows)):
            # print(len(first_inning_rows))
            row = second_inning_rows[i]
            cols = row.select('td')
            summary.append({
                'match': match_info,
                'teamInnings': team2,
                'battingPos': i+1,
                'battername': cols[0].select('a > span > span')[0].text.replace('\xa0', ''),
                'dismissal': cols[1].text,
                'runs': cols[2].select('strong')[0].text,
                'balls': cols[3].text,
                '4s': cols[5].text,
                '6s': cols[6].text,
                'SR': cols[7].text
            })
    except Exception as e:
        print(e)
    batting_summary.append(summary)

In [13]:
with open('../WPL_json_files/batting_summary.json', 'w') as f:
    json.dump(batting_summary, f, indent=4)